In [16]:
import numpy as np
import pandas as pd
import torch
import librosa
import os
import soundfile as sf

In [17]:
def clear_noise(waveform, alpha=2):        
    waveform = librosa.util.normalize(np.array(waveform))
    noisy_spectrum = np.abs(librosa.stft(waveform))        
    noise_spectrum = np.mean(noisy_spectrum, axis=1)
    clean_spectrum = np.maximum(noisy_spectrum - alpha * noise_spectrum[:, np.newaxis], 0)        
    clean_audio = librosa.istft(clean_spectrum)
    return torch.tensor(clean_audio)

In [20]:
df = pd.read_csv("file_info.csv")
df['path'] = df['path'].str.replace('dataset/', 'dataset_no_noize/')
df.head(5)

,path,label,sub_class
0,dataset_no_noize/dog/Dog/--7jZxfzemI.wav,dog,Dog
1,dataset_no_noize/dog/Dog/--CE2f-ttEQ.wav,dog,Dog
2,dataset_no_noize/dog/Dog/--ERHDSdxGQ.wav,dog,Dog
3,dataset_no_noize/dog/Dog/--L22BmDI6E.wav,dog,Dog
4,dataset_no_noize/dog/Dog/--sIMPsphRI.wav,dog,Dog


In [19]:
for index, row in df.iterrows():
    path = row['path']
    if os.path.exists(path):
        y, sr = librosa.load(path, sr=None)
        y = clear_noise(y)
        sf.write(path, y, sr)
    else:
        print(path, "Fail")

In [ ]:
df.to_csv('file_info.csv', index=False)